<a href="https://colab.research.google.com/github/dlskawns/RecSys_and_Retrieval_Study/blob/main/Memory_Based_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collaborative Filtering

## Memory Based Model - 메모리 기반 협업 필터링

유사도 기반 협업 필터링이라고도 불리는 이 메모리 기반 협업 필터링은 말 그대로 유저와 아이템 상호작용 기록에 의존하여 유저 또는 아이템 간의 유사도에 따라 상호작용이 없는 유저 또는 아이템에 대한 상호작용 점수를 예측하는 방식


In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd

# 데이터 불러오기
ratings = pd.read_table('/content/drive/MyDrive/ml-1m/ratings.dat', sep="::", header = None)
ratings = ratings.rename(columns = {0: 'userId',1:'movieId',2:'rating', 3:'time'})
print(ratings.shape)
ratings.head(5)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(1000209, 4)


,userId,movieId,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 유저, 영화의 평점 행렬로 변환
adj_matrix_rating = ratings.pivot(index = 'userId', columns = 'movieId', values = 'rating')

In [4]:
adj_matrix_rating

movieId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### KNN을 통해 비슷한 유저들을 바탕으로 특정 대상 선호도 파악하는 알고리즘

$\hat r_{u,i}$의 결측값을 0으로 두어 계산 진행



In [15]:
len(adj_matrix_rating)

6040

In [4]:
# for i in range(len(adj_matrix_rating)):
#   adj_matrix_rating.iloc[i]=adj_matrix_rating.iloc[i].fillna(adj_matrix_rating.iloc[i].mean())

adj_matrix_zero = adj_matrix_rating.fillna(0)

In [5]:
adj_matrix_zero

movieId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


$\hat r_{u,i} = \frac {\sum sim(u,u') \cdot (r_{u',i}) }{\sum sim(u,u')}$

Userbased

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

sim_u  = pd.DataFrame(cosine_similarity(adj_matrix_zero, adj_matrix_zero), columns = adj_matrix_zero.index)

In [7]:
sim_u[1].sort_values(ascending = False)[:10]

0       1.000000
5342    0.412117
5189    0.411899
1480    0.392110
1282    0.386597
5704    0.360898
5761    0.355600
1358    0.350329
1475    0.339496
540     0.334864
Name: 1, dtype: float64

In [8]:
def KNN_user(sim_matrix, user_num, k):
  """
  가장 유사한 이웃의 유저를 찾아 유저 번호를 리스트로 반환
  sim_matrix: 유저 유사도 매트릭스
  user_num: 유저 번호
  k: 이웃 수
  """
  lst = sim_matrix[user_num].sort_values(ascending = False)[:k]
  return lst
  

In [9]:
lst_2 = KNN_user(sim_u, 2, 25)
lst_2.index

Int64Index([   1, 3107,   94, 2813, 4600, 2302,  299, 3994, 3360, 4785,  557,
            4387,  809, 2499,  203, 5168, 2791, 2682, 3482, 3124,  998, 1695,
             645, 1779, 3598],
           dtype='int64')

In [10]:
# 유저가 본 영화 dictionary 
dict_u_i = {}
for i in adj_matrix_rating.index:
  dict_u_i.update({i: list(adj_matrix_rating.iloc[i-1].dropna().index)})

In [11]:


def none_movie_id(df, user_num):
  # 유저가 본 영화 딕셔너리
  dict_u_i = {}
  for i in df.index:
    dict_u_i.update({i: list(df.iloc[i-1].dropna().index)})

  # 유저가 안본영화 리스트
  idx_lst = []
  for i in df.columns:
    if i in dict_u_i[user_num]:
      pass
    else:
      idx_lst.append(i)
  return dict_u_i, idx_lst

## Cosine Similarity

$sim(x,y) = \frac {x \cdot y}{||x||^2_2 ||y||^2_2}$
= $\frac {x \cdot y}{ \sqrt {\sum x^2_i} \sqrt {\sum y^2_i}}$

In [20]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def Normal_KNN_Rec(sparse_matrix, user_num, k , weighted = True):
  """
  가장 유사한 이웃의 유저를 찾아 해당 유저를 바탕으로 낸 score 상위 10개 추천
  sparse_matrix: 유저-아이템 상호작용 매트릭스
  user_num: 유저 번호
  k: 이웃 수
  """
  # 결측치를 0으로 채워줌
  zero = sparse_matrix.fillna(0)
  
  # 결측치를 0으로 채운 값을 통해 유저 벡터 별 코사인 유사도를 구해 유저 유사도 매트릭스 구하기 (유저 x 유저)
  sim_matrix = pd.DataFrame(cosine_similarity(zero, zero), columns = sparse_matrix.index)

  # 유저가 안본 영화 구하기
  dic, nan_lst = none_movie_id(sparse_matrix, user_num)
  
  nan_dict ={}

  if weighted == True:
    for i in nan_lst:
      # 유저가 안본 영화를 본 다른 유저 구하기
      o_user = sparse_matrix[sparse_matrix[i].notnull()].index 

      # 그 유저중에 해당 유저와 가장 유사한 k명의 인덱스와 유사도를 가져온다. 
      lst = sim_matrix.iloc[o_user-1][user_num].sort_values(ascending = False)[:k]

      # k개 이웃의 유사도와 점수를 가져와 예상 점수를 계산한다.
      knn_sim = lst.values
      knn_idx = lst.index
      knn_score = sparse_matrix.iloc[knn_idx][i].values
      i_score = np.inner(knn_score, knn_sim)/ sum(knn_sim)
      nan_dict.update({i:i_score})
  if weighted == False:
    for i in nan_lst:
      # 유저가 안본 영화를 본 다른 유저 구하기
      o_user = sparse_matrix[sparse_matrix[i].notnull()].index 

      # 그 유저중에 해당 유저와 가장 유사한 k명의 인덱스와 유사도를 가져온다. 
      knn_idx = sim_matrix.iloc[o_user-1][user_num].sort_values(ascending = False)[:k].index

      # k개 이웃의 평균 점수를 가져와 예상 점수를 계산한다.
      i_score = sparse_matrix.iloc[knn_idx][i].mean()

      nan_dict.update({i:i_score})

  return sorted(nan_dict.items(), key = lambda item: item[1], reverse = True)[:11]
  

In [17]:
adj_matrix_rating.iloc[[2,10, 15]][2].mean()

nan

In [21]:
%%time

Normal_KNN_Rec(adj_matrix_rating, 2, 10, False)

CPU times: user 49.4 s, sys: 799 ms, total: 50.2 s
Wall time: 43.6 s


[(787, 5.0),
 (858, 5.0),
 (989, 5.0),
 (1830, 5.0),
 (3172, 5.0),
 (3233, 5.0),
 (3280, 5.0),
 (3382, 5.0),
 (3607, 5.0),
 (3656, 5.0),
 (3881, 5.0)]

In [22]:
%%time

Normal_KNN_Rec(adj_matrix_rating, 2, 10, True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


CPU times: user 49.4 s, sys: 796 ms, total: 50.2 s
Wall time: 42.4 s


[(787, 5.0),
 (989, 5.0),
 (1830, 5.0),
 (858, 4.999999999999999),
 (53, 4.810263423814847),
 (527, 4.80041138797396),
 (1234, 4.7003813717312966),
 (1221, 4.6052213127761705),
 (50, 4.6044516370398005),
 (1617, 4.603629063964186),
 (1272, 4.602895164916921)]

In [56]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
from tqdm import tqdm 

def scoring(df, k):
  
  df_pivot = df.pivot('userId','movieId','rating')
  df_zero = df_pivot.fillna(0)
  result = []
  for _, row in tqdm(df.iterrows()):

    user = row['userId']
    item = row['movieId']
    rating = row['rating']
    # 유저가 안본 영화를 본 다른 유저 구하기
    o_user = df_zero[df_zero[item]>0].index 

    # 그 유저중에 해당 유저와 가장 유사한 k명의 인덱스와 유사도를 가져온다. 
    lst = df_zero.loc[o_user][item].sort_values(ascending = False)[:k]

    # k개 이웃의 유사도와 점수를 가져와 예상 점수를 계산한다.
    knn_sim = lst.values
    knn_idx = lst.index

    knn_score = df_zero.loc[knn_idx][item].values

    i_score = np.inner(knn_score, knn_sim)/ sum(knn_sim)

    result.append([user, item, rating, i_score])
    
  return pd.DataFrame(result, columns = ['userId','itemId','rating','prediction'])

In [59]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ratings, test_size = 0.2, random_state = 2)

In [60]:
test.head(3)

,userId,movieId,rating,time
758856,4510,382,3,966801230
345912,2029,2797,3,974673768
586911,3585,919,5,966696267


In [107]:
test_df = scoring(test.sample(10000), 10)
np.sqrt(np.square(test_df['rating'] - test_df['prediction']).mean())

10000it [00:20, 476.30it/s]


0.9586789285503747

In [ ]:
test.pivot('userId','movieId','rating')

In [ ]:
def eval_rmse(df):
  
  df_pivot = df.pivot('userId','movieId','rating')
  df_zero = df_pivot.fillna(0)
  for _, row in df.iterrows():
    user = row['userId']
    item = row['itemId']
    rating = row['rating']
    

In [25]:
sorted(d.items(), key = lambda item: item[1], reverse= False)[:]

[(1, 1.3)]

In [99]:
import numpy as np
np.inner(sim_u, sim_u)

array([[105.92780989, 118.3015422 , 105.24408093, ...,  45.93121655,
         91.09964589, 146.83704223],
       [118.3015422 , 173.68800776, 139.26768893, ...,  57.96932343,
        109.23761532, 193.35656251],
       [105.24408093, 139.26768893, 133.90166188, ...,  53.39260039,
         98.33951298, 164.6733809 ],
       ...,
       [ 45.93121655,  57.96932343,  53.39260039, ...,  30.54018506,
         47.63263941,  75.57585177],
       [ 91.09964589, 109.23761532,  98.33951298, ...,  47.63263941,
        108.26131018, 153.8985086 ],
       [146.83704223, 193.35656251, 164.6733809 , ...,  75.57585177,
        153.8985086 , 266.98932235]])

In [114]:
knn_sim = sim_u.iloc[o_user][2].sort_values(ascending = False)[:25].values
knn_sim

array([0.36287498, 0.35531309, 0.34921693, 0.31852961, 0.31631181,
       0.3146523 , 0.31404352, 0.3130987 , 0.30939046, 0.30802892,
       0.3076401 , 0.30594372, 0.30561665, 0.30173366, 0.30111611,
       0.30079009, 0.30061745, 0.30058268, 0.29681088, 0.29555561,
       0.2951082 , 0.29406099, 0.29403379, 0.29399657, 0.29311021])

In [116]:
knn_idx = sim_u.iloc[o_user][2].sort_values(ascending = False)[:25].index
knn_idx

Int64Index([4600, 3994, 4387, 5682, 4888, 4006,  549, 5530, 5619, 1854,  777,
             337, 4041, 3756, 3624,  476, 1120, 3539,  146, 3259, 4788, 4830,
             147, 1339, 1087],
           dtype='int64')

In [119]:
knn_score = adj_matrix_rating.iloc[knn_idx-1][2].values

In [121]:
np.inner(knn_score, knn_sim)/sum(knn_sim)

3.2803592894787164

In [ ]:
dict_u_i[1]

In [ ]:
def prediction(df, sim_matrix, user_num, k, ):
  knn = KNN_user(sim_matrix, user_num, k)